In [3]:
# !pip install langchain
# !pip install langchain-openai
!pip install langchain-community

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.23.1-py3-none-any.whl.metadata (7.5 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.2 MB/s eta 0:00:00
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached marshmallow-3.23.1-py3-none-any.whl (49 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)


In [5]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from dotenv import load_dotenv

In [7]:
load_dotenv()

True

In [9]:
model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)

/var/folders/9y/bw4br9h516s_4pbmq21pjhl40000gn/T/ipykernel_2569/3737214807.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.5)


### A simple string based Prompt formatting

In [12]:
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

prompt = prompt_template.format(adjective = "good", content="Hard Work")

response = model.invoke(prompt)
print(response)



Why was the hard-working ant always stressed out?

Because he had too many deadlines to meet!


### ChatPromptTemplate: The prompt to chat models is a list of chat messages

In [15]:
from langchain_core.prompts import ChatPromptTemplate

In [18]:
chat_template = ChatPromptTemplate.from_messages(
[
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai","I'm doing well, thanks!"),
    ("human", "{user_input}")
]
)

prompt = chat_template.format_messages(name="Bob", user_input = "What do you say when a person is not just focussed on results but gettings things done and knowing more?")
response = model.invoke(prompt)
print(response)


AI: I would say that person is a well-rounded and efficient individual who values both productivity and knowledge.


### Various ways of formatting System/Human/AI Prompts

In [21]:
chat_template = ChatPromptTemplate.from_messages(
    [SystemMessage(
        content = (
            "You are a helpful assistant that re-writes the user's text to "
            "Sound more upbeat."
        )
    ),
     HumanMessagePromptTemplate.from_template("{text}")
    ]
)

prompt = chat_template.format_messages(text = "I dont like to be lazy although I am super lazy")

response = model.invoke(prompt)
print(response)


System: I may have a tendency to procrastinate, but I am also incredibly efficient and productive!


### Providing a context along with the prompt

In [24]:
prompt = """
 Answer the question based on the context below. If the question cannot be answered using the information provided answer with I don't know.


 Context" Large Language Models (LLMs) are the latest models used in NLP. 
 Their superior performance over smaller models has made them incredibly
 useful for developers building NLP enabled applications. These models
 can be accessed via Hugging Face's 'transformers' library, via OpenAI using 'openai' library
 , and via Cohere using the 'cohere' library.

 Question: Which libraries and model provideers offer LLMs?
 
Answer: """


print(model.invoke(prompt))

Hugging Face, OpenAI, and Cohere offer LLMs.


In [26]:
prompt = """
 Answer the question based on the context below. If the question cannot be answered using the information provided answer with I don't know.


 Context" Large Language Models (LLMs) are the latest models used in NLP. 
 Their superior performance over smaller models has made them incredibly
 useful for developers building NLP enabled applications. These models
 can be accessed via Hugging Face's 'transformers' library, via OpenAI using 'openai' library
 , and via Cohere using the 'cohere' library.

 Question: Who is Cristiano Ronaldo?
 
Answer: """


print(model.invoke(prompt))

I don't know.


### Few Shot Prompt Template | LangChain
The goal of few shot prompt templates are to dynamically select examples based on an input, and then format the examples in a final prompt to provide for the model.

#### Fixed Examples
The most basic and common few-shot prompting technique is to use a fixed prompt example. This way, we can select a chain, evaluate it, and avoid worrying about additional moving parts in production.

In [29]:
from langchain import FewShotPromptTemplate

examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do"
    },
    {
        "query": "What time is it?",
        "answer": "It is time to buy a new watch"
    }
]

example_template = """
User: {query}
AI: {answer}
"""

In [31]:
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template

)


prefix = """
The following are exerpts from conversations with an AI assistant. The assistant is sarcastic and witty, producing creative
, and funny responses to the users questions. Here are some examples:
"""


suffix = """
User: {query}
AI: 
"""


In [33]:
few_shot_prompt_template = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    prefix = prefix,
    suffix = suffix,
    input_variables = ["query"],
    example_separator = "\n\n"
)


query = "Should I go and play football even though I have to study?"

print(few_shot_prompt_template.format(query=query))


The following are exerpts from conversations with an AI assistant. The assistant is sarcastic and witty, producing creative
, and funny responses to the users questions. Here are some examples:



User: How are you?
AI: I can't complain but sometimes I still do



User: What time is it?
AI: It is time to buy a new watch



User: Should I go and play football even though I have to study?
AI: 



In [37]:
chain = few_shot_prompt_template | model

chain.invoke({"query":query})

'Oh sure, because who needs good grades when you can have a sprained ankle and a concussion?'